In [1]:
# Import libraries
# import pandas as pd
import pickle

import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# import seaborn as sns
import cv2

from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# gpu settings
# Ref: https://www.tensorflow.org/guide/gpu
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# a list of all the labels we have
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
          'X', 'Y']
num_of_image = 500  # we have 100 images for each label
image_resolution = 128
input_dir_path = './asl_gray/'

X_train = []
Y_train = []
# read grayscaled images
for i in range(len(labels)):
    for j in range(num_of_image):
        temp_im = cv2.imread(input_dir_path + labels[i] + '/' + labels[i] + str(j) + '.png', cv2.IMREAD_COLOR)
        # temp_im = 255*temp_im/np.max(temp_im)
        temp_im = cv2.resize(temp_im, (image_resolution, image_resolution))
        X_train.append(temp_im)
        Y_train.append(i)
        
# convert to a 1200x24 matrix
# where if label = i, then the row have 1 in ith column, and 0 otherwise
Y_train = to_categorical(Y_train, num_classes=24)

# normalize the data
X_train = np.array(X_train) / 255
X_train = X_train.reshape(-1, image_resolution, image_resolution, 3)
# print(X_train[0].shape)
# cross-validation: 9:1
X_t, X_v, Y_t, Y_v = train_test_split(X_train, Y_train, test_size=0.3, random_state=0, shuffle=True)

In [4]:
# The model
model = Sequential()
# Convolution layers
# Layer 1
model.add(
    Conv2D(filters=32, kernel_size=(5, 5), padding='Same', activation='relu', input_shape=(image_resolution, image_resolution, 3), use_bias=True, strides=(2, 2)))
model.add(MaxPool2D(pool_size=(2, 2)))  # downsampling
model.add(Dropout(0.25))  # Dropout reduces overfitting
# Layer 2
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu', use_bias=True))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

# Layer 3
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='Same', activation='relu', use_bias=True))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))
#
# # Layer 4
# model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='Same', activation='relu',use_bias=True))
# model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Dropout(0.25))

# Fully connected layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(24, activation='softmax'))

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# use categorical crossentropy as loss function
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define model parameters
epochs = 30
batch_size = 40
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       7

In [5]:

model.fit(X_t, Y_t, batch_size=batch_size, epochs=epochs, validation_data=(X_v,Y_v), callbacks=[learning_rate_reduction])
# final = model.fit(X_train2, Y_train2, batch_size=batch_size, epochs=epochs, validation_data=(X_val,Y_val), callbacks=[learning_rate_reduction])


Epoch 1/30
210/210 [==============================] - 6s 18ms/step - loss: 3.3738 - accuracy: 0.0968 - val_loss: 2.6226 - val_accuracy: 0.2269
Epoch 2/30
 13/210 [>.............................] - ETA: 2s - loss: 2.4851 - accuracy: 0.2483

F:\ANACONDA\envs\env1\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


210/210 [==============================] - 3s 16ms/step - loss: 2.2825 - accuracy: 0.3081 - val_loss: 3.5489 - val_accuracy: 0.1325
Epoch 3/30
210/210 [==============================] - 3s 15ms/step - loss: 1.5307 - accuracy: 0.5079 - val_loss: 1.5242 - val_accuracy: 0.5156
Epoch 4/30
210/210 [==============================] - 3s 15ms/step - loss: 1.0696 - accuracy: 0.6529 - val_loss: 0.9081 - val_accuracy: 0.7103
Epoch 5/30
210/210 [==============================] - 3s 15ms/step - loss: 0.7633 - accuracy: 0.7544 - val_loss: 0.6873 - val_accuracy: 0.7828
Epoch 6/30
210/210 [==============================] - 3s 16ms/step - loss: 0.6074 - accuracy: 0.8052 - val_loss: 0.4086 - val_accuracy: 0.8511
Epoch 7/30
210/210 [==============================] - 3s 16ms/step - loss: 0.4657 - accuracy: 0.8532 - val_loss: 0.3775 - val_accuracy: 0.8828
Epoch 8/30
210/210 [==============================] - 3s 16ms/step - loss: 0.3732 - accuracy: 0.8778 - val_loss: 0.7084 - val_accuracy: 0.7711
Epoch 9/30

In [6]:
test_path = './asl_alphabet_test/'
X_test = []
Y_test = []
for i in range(len(labels)):
    temp_im = cv2.imread(test_path + labels[i] +'_test.jpg')
    temp_im = cv2.resize(temp_im, (image_resolution, image_resolution))
    X_test.append(temp_im)
    Y_test.append(i)

Y_test = to_categorical(Y_test, num_classes=24)

# normalize the data
X_test = np.array(X_test) / 255
X_test = X_test.reshape(-1, image_resolution, image_resolution, 3)

In [7]:

# Get CNN loss and test error
model.evaluate(X_test, Y_test)

1/1 [==============================] - 0s 147ms/step - loss: 3.5741e-04 - accuracy: 1.0000


[0.00035740784369409084, 1.0]

In [8]:
import matplotlib.pyplot as plt

plt.plot(model.history['accuracy'])
plt.plot(model.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Epochs')
plt.xlabel('Accuracy')
plt.legend(['Train, Validation'], loc = 'best')
plt.show()
plt.plot(model.history['loss'])
plt.plot(model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Trian', 'Validation'], loc='upper left')
plt.show()

TypeError: 'History' object is not subscriptable

In [9]:
json_mod = model.to_json()
with open('cnn_model_500im_epoch30_batch40_layer3_RGB_reshape.json', 'w') as file:
    file.write(json_mod)
model.save_weights("model.h5")
print("Saved")

Saved
